In [1]:
import tensorflow as tf

2023-11-16 22:10:00.470334: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-16 22:10:00.470354: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-16 22:10:00.470358: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-16 22:10:00.473367: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [79]:
import tensorflow.keras as keras

class _TODO_BatchIntervalSampling(keras.layers.Layer):
    def __init__(self, num_samples, input_axis=0, **kwargs):
        super().__init__(**kwargs)
        self.num_samples = num_samples
        self.input_axis = input_axis
    
    def call(self, inputs):
        # TODO ref https://stackoverflow.com/a/64436208
        def _impl_array_slice(
            a, axis, 
            start, stop, 
            step=1,
        ):
            return a[
                (slice(None),) * (axis % a.ndim) 
                + (slice(start, stop, step),)
            ]
        
        def _impl_array_sample(
            a, axis, 
            n_samples,
            start, stop, 
        ):
            # NOTE first + n * step == last
            _aligned_interval = lambda first, last, n: (
                first,
                last - (last - first) % n,
                (last - first) // n,
            )

            first, last, step = _aligned_interval(
                start, stop - 1, 
                # NOTE number of intervals
                n=n_samples - 1
            )
        
            return _impl_array_slice(
                a, axis=axis,
                start=first, stop=last + 1,
                step=step,
            )

        def _array_sample(
            a, axis, 
            n_samples,
            start=None, stop=None, 
        ):
            if start is None: start = 0
            if stop is None: stop = a.shape[axis]
            return _impl_array_sample(
                a, axis, 
                n_samples=n_samples,
                start=start, stop=stop,
            )

        # TODO ratio
        return tf.nest.map_structure(
            lambda input: _array_sample(
                input, axis=self.input_axis,
                n_samples=self.num_samples,
            ), 
            inputs,
        )

In [78]:



def _TODO_rm_array_sample(a, axis, step=1):
    return _array_slice(
        a, axis=axis,
        start=0,
        stop=a.shape[axis],
        step=step,
    )


a = tf.stack([
        tf.range(0, 13),
        tf.range(0, 13),
    ])
axis = 1

_TODO_rm_array_sample(
    a,
    axis=axis,
    step=2,
)

_array_sample(
    a,
    axis=axis,
    n_samples=2,
)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 0, 12],
       [ 0, 12]], dtype=int32)>

In [ ]:
# last = stop - 1
# first + num * interval = last

In [53]:
r = tf.range(0, 13)


In [73]:
for num in range(1, 13):
    first, last = r[0], r[-1]
    n_intervals = num - 1
    last -= (last - first) % n_intervals
    step = (last - first) // n_intervals
    
    print(num, r[first:last + 1:step].shape)

1 (2,)
2 (2,)
3 (3,)
4 (4,)
5 (5,)
6 (6,)
7 (7,)
8 (8,)
9 (9,)
10 (10,)
11 (11,)
12 (12,)


In [ ]:
_aligned_interval = lambda first, last, n: (
    first,
    last - (last - first) % n,
    (last - first) // n,
)

In [66]:
(last - first) / n_intervals

<tf.Tensor: shape=(), dtype=float64, numpy=2.0>

In [62]:
last

<tf.Tensor: shape=(), dtype=int32, numpy=10>

In [50]:
0 + 4 * 3

12

In [37]:
tf.range(0, 13)

<tf.Tensor: shape=(13,), dtype=int32, numpy=array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12], dtype=int32)>

In [ ]:
0 1 2 3 4 5

In [35]:
13/4

3.25

In [32]:
13/5

2.6

In [28]:
13//5

2

In [ ]:
#0 3 6 9 12

In [24]:
tf.range(0, 13)

<tf.Tensor: shape=(13,), dtype=int32, numpy=array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12], dtype=int32)>

In [13]:
tf.stack([
    tf.range(0, 10),
    tf.range(0, 10),
]).ndim #[slice(None), 0:9:2]

2

InvalidArgumentError: {{function_node __wrapped__Pack_N_2_device_/job:localhost/replica:0/task:0/device:GPU:0}} Shapes of all inputs must match: values[0].shape = [10] != values[1].shape = [12] [Op:Pack] name: stack